### Challenge 03.
### Sudoku Solver.
### *[LeetCode 37.](https://leetcode.com/problems/sudoku-solver/)*

##### **Description**
Write a program to solve a Sudoku puzzle by filling the empty cells.

A sudoku solution must satisfy all of the following rules:

Each of the digits 1-9 must occur exactly once in each row.
Each of the digits 1-9 must occur exactly once in each column.
Each of the digits 1-9 must occur exactly once in each of the 9 3x3 sub-boxes of the grid.
The '.' character indicates empty cells.

##### **Examples** 
*Example 1:*

Input: board = \
[["5","3",".",".","7",".",".",".","."],\
["6",".",".","1","9","5",".",".","."],\
[".","9","8",".",".",".",".","6","."],\
["8",".",".",".","6",".",".",".","3"],\
["4",".",".","8",".","3",".",".","1"],\
["7",".",".",".","2",".",".",".","6"],\
[".","6",".",".",".",".","2","8","."],\
[".",".",".","4","1","9",".",".","5"],\
[".",".",".",".","8",".",".","7","9"]]

Output: \
[["5","3","4","6","7","8","9","1","2"],\
["6","7","2","1","9","5","3","4","8"],\
["1","9","8","3","4","2","5","6","7"],\
["8","5","9","7","6","1","4","2","3"],\
["4","2","6","8","5","3","7","9","1"],\
["7","1","3","9","2","4","8","5","6"],\
["9","6","1","5","3","7","2","8","4"],\
["2","8","7","4","1","9","6","3","5"],\
["3","4","5","2","8","6","1","7","9"]]

### Proposed Solution

In [4]:
### Challenge 03
### Sudoku Solver
### Proposed Solution



### Import libraries
import numpy as np

### Load functions from Challenge 02

# Load and check input
def load_board(input):

    # Convert to numeric list
    for ii in range(len(input)):
        for jj in range(len(input[ii])):
            # Replace "." by np.nan
            if input[ii][jj] == ".":
                input[ii][jj] = np.nan
            # Otherwise cast to integer
            else:
                input[ii][jj] = float(input[ii][jj])

    # Create matrix from numeric list
    main_array = np.array(input, dtype=float)

    # Check input size
    mas = main_array.shape
    nrows = mas[0]
    ncolumns = mas[1]
    if (nrows == 9) and (ncolumns == 9):
        check = True
    else:
        check = False

    # Result
    return main_array, check

# Building block for validations
def nine_elements_validation(array):
    aux_no_nan = np.sort(array[~np.isnan(array)])
    aux_unique = np.array(list(set(aux_no_nan)))
    if len(aux_no_nan) == len(aux_unique):
        validation = True
    else:
        validation = False
    return validation

# Validation over rows and columns
def valid_straight_lines(main_array, axis="rows"):
    ls_validation = [None]*9
    for ii in range(9):
        if axis == "columns":
            aux = main_array[ii][:]
        else: 
            aux = main_array[:][ii]
        ls_validation[ii] = nine_elements_validation(aux)
    return all(ls_validation)

# Validation over 3x3 sub-boxes
def valid_subboxes(main_array):
    ls_validation = [None]*9
    for ii in range(3):
        for jj in range(3):
            subbox = main_array[3*ii:3*(ii+1), 3*jj:3*(jj+1)]
            rs_subbbox = subbox.reshape(9)
            ls_validation[(3*ii)+jj] = nine_elements_validation(rs_subbbox)
    return all(ls_validation)


# Entire validation
def vs(input):
    main_array, check = load_board(input)
    if check == True:
        vrows = valid_straight_lines(main_array,"rows")
        vcols = valid_straight_lines(main_array,"columns")
        vboxs = valid_subboxes(main_array)
        entire_validation = all([vrows,vcols,vboxs])
        return {"input_allowed": entire_validation,
                "valid_rows": vrows,
                "valid_columns": vcols,
                "valid_subboxes": vboxs}
    else:
        return {"input_allowed": False, 
                "message": "The input has to be a list of lists, representing a 9x9 sudoku block."}


In [6]:
### Challenge 03
### Sudoku Solver
### Proposed Solution


### Definition of functions

# First step function: sweep every cell to find its correct value
# It takes into account row, column, and subbox neighbors of each cell
def first_step(main_array):

    ref_set = {1, 2, 3, 4, 5, 6, 7, 8, 9}
    for ii in range(9):
        for jj in range(9):

            # Check if the value [ii][jj] is empty
            if np.isnan(main_array[ii,jj]):
            
                # Get row, column, and subbox related to the value [ii][jj]
                aux_row = main_array[ii,:]
                aux_col = main_array[:,jj]
                quo_ii, quo_jj = int(ii/3), int(jj/3)
                aux_sbx = main_array[3*quo_ii:3*(quo_ii+1), 3*quo_jj:3*(quo_jj+1)]
                
                # Flatten subbox array
                aux_sbx_flat = aux_sbx.reshape(9)
                
                # Concatenate three auxiliary arrays
                all_aux_arrays = np.concatenate((aux_row, aux_col, aux_sbx_flat), axis=None)
                
                # Array having numbers already occupied
                ar_no_nan = np.sort(all_aux_arrays[~np.isnan(all_aux_arrays)]).astype(int)
                
                # List of possible options for the value [ii][jj]
                possible_values = list(ref_set.difference(set(ar_no_nan)))
                
                # If there is only one possible value assign it to [ii][jj]
                if len(possible_values) == 1:
                    main_array[ii,jj] = possible_values[0]
    
    return main_array



# Auxiliary (I) function for second step function
# Get missing values from a subbox
def possible_values_in_subbox(main_array, row_block, col_block):

    # Redefine subbox parameters
    ii, jj = row_block, col_block    
    # Set of reference
    ref_set = {1, 2, 3, 4, 5, 6, 7, 8, 9}
    # Get subbox
    aux_sbx = main_array[3*ii:3*(ii+1), 3*jj:3*(jj+1)]
    # Flatten subbox array
    aux_sbx_flat = aux_sbx.reshape(9)
    # Array having numbers already occupied
    ar_no_nan = np.sort(aux_sbx_flat[~np.isnan(aux_sbx_flat)]).astype(int)
    # List of needed values inside the subbox
    possible_values = list(ref_set.difference(set(ar_no_nan)))
    
    return possible_values



# Auxiliary (II) function for second step function
# Assign value taking into account its presence in neighbor subboxes
def assign_value_into_empty_subbox_position(main_array, row_block, col_block, empty_indexes_sbx, value):

    # Redefine subbox parameters
    ii, jj = row_block, col_block

    # Search for the value row-wise within subboxes neighbors
    three_subboxes_rowwise = main_array[3*ii:3*(ii+1),:]
    aux_row_indices_to_block = np.argwhere(three_subboxes_rowwise==value)
    if len(aux_row_indices_to_block) > 0:
        # Block rows from list of empty positions
        row_indices_to_block = [(3*ii)+index[0] for index in aux_row_indices_to_block]
        empty_indexes_sbx = [elem for elem in empty_indexes_sbx if elem[0] not in row_indices_to_block]

    # Search for the value column-wise within subboxes neighbors
    three_subboxes_colwise = main_array[:,3*jj:3*(jj+1)]
    aux_col_indices_to_block = np.argwhere(three_subboxes_colwise==value)
    if len(aux_col_indices_to_block) > 0:
        # Block columns from list of empty positions
        col_indices_to_block = [(3*jj)+index[1] for index in aux_col_indices_to_block]
        empty_indexes_sbx = [elem for elem in empty_indexes_sbx if elem[1] not in col_indices_to_block]

    # Assign the value if only one position is allowed within the subbox
    if len(empty_indexes_sbx) == 1:
        main_array[empty_indexes_sbx[0][0], empty_indexes_sbx[0][1]] = value

    return main_array


# Second step function: find possible values within a subbox
# It takes into account already filled values from neighbor subboxes 
def second_step(main_array):

    for ii in range(3):
        for jj in range(3):
            
            # Call function 'possible_values_in_subbox'
            possible_values = possible_values_in_subbox(main_array, row_block=ii, col_block=jj)

            # Check if there are missing values in the subbox
            if len(possible_values) > 0:

                # Iterate over all possible values
                for value in possible_values:   

                    # List of empty positions in the subbox
                    aux_empty_indexes = np.argwhere(np.isnan(main_array[3*ii:3*(ii+1), 3*jj:3*(jj+1)]))
                    empty_indexes_sbx = np.array([[(3*ii)+elem[0], (3*jj)+elem[1]] for elem in aux_empty_indexes])

                    # Call function 'assign_value_into_empty_subbox_position'
                    main_array = assign_value_into_empty_subbox_position(main_array, row_block=ii, col_block=jj, empty_indexes_sbx=empty_indexes_sbx, value=value)

    return main_array


# Auxiliary (I) function for third step function
# Get all the available positions in the board for a specific value
def available_indexes_for_value(main_array, value):

    # Empty positions
    aux_empty_indexes = np.argwhere(np.isnan(main_array))
    empty_indexes = np.array([[elem[0],elem[1]] for elem in aux_empty_indexes])

    # Prohibited positions for value (row-wise and column-wise)
    aux_positions_to_block = np.argwhere(main_array==value)
    if len(aux_positions_to_block) > 0:
        for elem_block in aux_positions_to_block:
            # Delete prohibited positions from empty positions
            empty_indexes = [ei for ei in empty_indexes if (ei[0] != elem_block[0]) and (ei[1] != elem_block[1])]

    # Prohibited positions for value (subbox)
    for ii in range(3):
        for jj in range(3):
            possible_values = possible_values_in_subbox(main_array, row_block=ii, col_block=jj)
            
            # Prohibited positions for value in subboxes that already had the value
            if value not in possible_values:
                # Delete prohibited positions from empty positions
                empty_indexes = [ei for ei in empty_indexes if ((ei[0] >= (3*ii)) and (ei[0] < 3*(ii+1)) and (ei[1] >= (3*jj)) and (ei[1] < 3*(jj+1))) == False]

    # Prohibited positions for value (subbox neighbors)
    for ii in range(3):
        for jj in range(3):

            # Prohibited positions for value due to prohibitions from neighbor subboxes
            empty_indexes_subbox = [[ei[0], ei[1]] for ei in empty_indexes if ((ei[0] >= (3*ii)) and (ei[0] < 3*(ii+1)) and (ei[1] >= (3*jj)) and (ei[1] < 3*(jj+1))) == True]
            rows_to_block = list(set([ei[0] for ei in empty_indexes_subbox]))
            unique_row = len(rows_to_block)==1
            if unique_row:
                # Delete prohibited positions from empty positions
                empty_indexes = [ei for ei in empty_indexes if (ei[0] != rows_to_block[0]) or ((ei[1] >= 3*jj) and (ei[1] < 3*(jj+1)))]
            
            cols_to_block = list(set([ei[1] for ei in empty_indexes_subbox]))
            unique_col = len(cols_to_block)==1
            if unique_col:
                # Delete prohibited positions from empty positions
                empty_indexes = [ei for ei in empty_indexes if (ei[1] != cols_to_block[0]) or ((ei[0] >= 3*ii) and (ei[0] < 3*(ii+1)))]

    return empty_indexes



# Auxiliary (II) function for third step function
# Generalization of function 'available_indexes_for_value'
def find_possible_locations_for_all_values(main_array):

    # Dictionary of possible positions for each value
    empty_indexes_by_value = dict()
    for value in range(1,10):
        empty_indexes_by_value[value] = available_indexes_for_value(main_array, value=value)

    # Find two numbers that share the same two possible positions within a subbox
    for first_value in range(1,10):
        for second_value in range(1,first_value):
            empty_indexes_by_value_a = empty_indexes_by_value[first_value]
            empty_indexes_by_value_b = empty_indexes_by_value[second_value]

            # Iterate over subboxes
            for ii in range(3):
                for jj in range(3):

                    # Empty indexes within a subbox
                    eibva_in_subbox = np.array([elem for elem in empty_indexes_by_value_a if (elem[0]>=3*ii) and (elem[0]<3*(ii+1)) and (elem[1]>=3*jj) and (elem[1]<3*(jj+1))])
                    eibvb_in_subbox = np.array([elem for elem in empty_indexes_by_value_b if (elem[0]>=3*ii) and (elem[0]<3*(ii+1)) and (elem[1]>=3*jj) and (elem[1]<3*(jj+1))])

                    # Compare arrays of empty indexes and check they only have 2 elements
                    if (np.array_equal(eibva_in_subbox, eibvb_in_subbox)) and (len(eibva_in_subbox)==2):

                        # Block column within subbox
                        cols_to_block = list(set([elem[1] for elem in eibva_in_subbox]))
                        if len(cols_to_block) == 1:
                            for third_value in range(1,10):
                                if (third_value != first_value) and (third_value != second_value):
                                    empty_indexes_by_value[third_value] = [ei for ei in empty_indexes_by_value[third_value] if ((ei[1] == cols_to_block[0]) and (ei[0]>=3*ii) and (ei[0]<3*(ii+1)))==False]

                        # Block row within subbox
                        rows_to_block = list(set([elem[0] for elem in eibva_in_subbox]))
                        if len(rows_to_block) == 1:
                            for third_value in range(1,10):
                                if (third_value != first_value) and (third_value != second_value):
                                    empty_indexes_by_value[third_value] = [ei for ei in empty_indexes_by_value[third_value] if ((ei[0] == rows_to_block[0]) and (ei[1]>=3*jj) and (ei[1]<3*(jj+1)))==False]

    return empty_indexes_by_value



# Third step function: finds the possible values within a subbox 
# It is based on prohibitions from neighbor subboxes
def third_step(main_array):

    # Call function 'available_indexes_for_value'
    empty_indexes_by_value = find_possible_locations_for_all_values(main_array)

    # Iterate over all possible values
    for value in range(1,10):

        # Get empty indexes for value
        empty_indexes = empty_indexes_by_value[value]

        # Assign value if the subbox has only one available position
        for ii in range(3):
            for jj in range(3):
                empty_indexes_subbox = [[ei[0], ei[1]] for ei in empty_indexes if ((ei[0] >= (3*ii)) and (ei[0] < 3*(ii+1)) and (ei[1] >= (3*jj)) and (ei[1] < 3*(jj+1))) == True]
                if len(empty_indexes_subbox)==1:
                    main_array[empty_indexes_subbox[0][0], empty_indexes_subbox[0][1]] = value

        # Alternative assignation based one prohibition of rows
        for ii in range(3):
            for row in range(3*ii,3*(ii+1)):
                empty_indexes_neighbors = [[ei[0], ei[1]] for ei in empty_indexes if ei[0] == row]
                if len(empty_indexes_neighbors)==1:
                    main_array[empty_indexes_neighbors[0][0], empty_indexes_neighbors[0][1]] = value
        
        # Alternative assignation based one prohibition of rows
        for jj in range(3):
            for col in range(3*jj,3*(jj+1)):
                empty_indexes_neighbors = [[ei[0], ei[1]] for ei in empty_indexes if ei[1] == col]
                if len(empty_indexes_neighbors)==1:
                    main_array[empty_indexes_neighbors[0][0], empty_indexes_neighbors[0][1]] = value

    return main_array



# Fourth step function: finds the possible values for a specific position
# It is derived from a dictionary whose key:value stands for empty_position:possible_values
def fourth_step(main_array):

    # Call function 'find_possible_locations_for_all_values'
    empty_indexes_by_value = find_possible_locations_for_all_values(main_array)

    # Transform dictionary 'empty_indexes_by_value' into 'possible_values_by_indexes'
    possible_values_by_indexes = dict()
    for key, value in empty_indexes_by_value.items():
        for position in value:
            new_key = str(position[0])+"_"+str(position[1])
            if new_key not in list(possible_values_by_indexes.keys()):
                possible_values_by_indexes[new_key] = [int(key)]
            else:
                possible_values_by_indexes[new_key] = possible_values_by_indexes[new_key] + [int(key)]

    # Assign value based on dictionary 'possible_values_by_indexes'
    for key, value in possible_values_by_indexes.items():
        ii, jj = int(key[0]), int(key[2])
        row_block = int(ii/3)
        col_block = int(jj/3)
        possible_values = possible_values_in_subbox(main_array, row_block, col_block)
        if (len(value)==1) and (value[0] in possible_values):
            main_array[ii,jj] = value[0]

    return main_array



# Solution of the Sudoku via iterations (main function)
def main(input):

    # Check the validity of the input
    valid_input = vs(input)

    # Case when the input is valid
    if valid_input["input_allowed"] == True:

        # Initialize variable for number of iterations
        number_iterations = 0

        # Initialize dictionary for saving the history of the solution
        dc_solution_history = dict()

        # Empty cells are counted for the first time
        main_array = load_board(input)[0]
        main_array_flat = main_array.reshape(81)
        number_empty_cells = len(list(main_array_flat[np.isnan(main_array_flat)]))

        # Iterations of the different step functions
        while number_empty_cells > 1:

            # Increase the number of iterations
            number_iterations += 1

            # First step
            main_array = first_step(main_array)
            current_board_status = np.copy(main_array)
            new_key = "move_"+str((4*(number_iterations-1))+1)
            dc_solution_history[new_key] = {"iteration_number":number_iterations, "step":"first", "board_state":current_board_status}
            
            # Second step
            main_array = second_step(main_array)
            current_board_status = np.copy(main_array)
            new_key = "move_"+str((4*(number_iterations-1))+2)
            dc_solution_history[new_key] = {"iteration_number":number_iterations, "step":"second", "board_state":current_board_status}
            
            # Third step
            main_array = third_step(main_array)
            current_board_status = np.copy(main_array)
            new_key = "move_"+str((4*(number_iterations-1))+3)
            dc_solution_history[new_key] = {"iteration_number":number_iterations, "step":"third", "board_state":current_board_status}
            
            # Fourth step
            main_array = fourth_step(main_array)
            current_board_status = np.copy(main_array)
            new_key = "move_"+str((4*(number_iterations-1))+4)
            dc_solution_history[new_key] = {"iteration_number":number_iterations, "step":"fourth", "board_state":current_board_status}

            # Empty cells counted after each iteration
            main_array_flat = main_array.reshape(81)
            number_empty_cells = len(list(main_array_flat[np.isnan(main_array_flat)]))

            # Emergency break (the solution should not require more than 10 iterations)
            if number_iterations==10:
                print("Emergency break was applied")
                break

        # Solution 
        result = {"original_board": load_board(input)[0],
                  "valid_input": valid_input,
                  "solved": True,
                  "solution": main_array,
                  "solution_history": dc_solution_history,
                  "number_iterations": number_iterations}
    
    # Case when the input is not valid
    else:
        result = {"original_board": load_board(input)[0],
                  "valid_input": valid_input,
                  "solved": False}

    return result

In [8]:
### Testing Proposed Solution
### Execution of the Sudoku's solution (main function)

# Initialize inputs
board_1 = [["5","3",".",".","7",".",".",".","."]
,["6",".",".","1","9","5",".",".","."]
,[".","9","8",".",".",".",".","6","."]
,["8",".",".",".","6",".",".",".","3"]
,["4",".",".","8",".","3",".",".","1"]
,["7",".",".",".","2",".",".",".","6"]
,[".","6",".",".",".",".","2","8","."]
,[".",".",".","4","1","9",".",".","5"]
,[".",".",".",".","8",".",".","7","9"]]
board_2 = [[".",".",".",".",".",".",".",".","."]
,["7",".",".",".","8",".",".",".","5"]
,["2",".",".","9",".",".","3",".","."]
,["9",".",".","7",".",".",".",".","8"]
,["4","7",".","3",".",".",".","5","."]
,[".",".",".","4",".",".",".",".","1"]
,["5",".",".",".","6",".",".",".","2"]
,[".","1","4",".",".",".",".",".","."]
,[".",".","2",".","9",".",".","1","."]]
board_3 = [["5","3",".",".","7",".",".",".","."]
,["6",".",".","1","9","5",".",".","."]
,[".","9","8",".",".",".",".","6","."]
,["8",".",".",".","6",".",".",".","3"]
,["4",".",".","8",".","3",".",".","1"]
,["7",".",".",".","2",".",".",".","6"]
,[".","6",".",".",".",".","2","8","."]
,[".",".",".","4","1","9",".",".","5"]
,[".",".",".",".","8",".",".","7","9"]]
board_4 = [["2","5",".",".",".",".",".",".","4"]
,[".",".",".",".","5",".",".",".","9"]
,[".","8",".","3",".",".","2","5","."]
,[".",".",".",".",".",".",".",".","2"]
,[".","3",".",".",".","7",".",".","."]
,["8",".",".",".","4",".","1","6","."]
,["1",".",".",".","6",".","5","8","."]
,[".",".",".",".",".",".",".","9","."]
,[".",".","6","4",".",".",".",".","."]]
board_5 = [[".","4",".",".","1","9",".","7","6"]
,["8",".",".",".",".",".",".",".","3"]
,[".",".",".","6",".",".",".",".","."]
,[".","9",".",".","2","7",".","1","."]
,[".",".","4",".",".",".","9",".","."]
,[".",".",".",".",".","5",".",".","."]
,[".",".","3",".","6","2",".",".","7"]
,[".","2",".","5",".",".",".",".","."]
,[".",".",".","4",".",".",".","6","."]]
board_6 = [[".",".","1",".",".","4","8",".","."]
,[".",".","3","2","8",".",".",".","5"]
,[".","2",".",".",".","6",".",".","."]
,[".",".","5",".",".",".",".","7","."]
,[".","3",".","9","1",".","6",".","."]
,[".",".",".",".",".","2",".",".","."]
,[".","9",".","8","3",".","1",".","."]
,["1",".",".",".",".",".",".",".","6"]
,[".",".",".",".","4",".",".",".","."]]
board_7 = [["1",".",".",".","6",".",".",".","."]
,[".",".","3","9",".","1",".","4","."]
,["2",".",".",".",".",".",".",".","7"]
,[".",".",".",".","8",".",".","5","."]
,[".",".","6",".","4",".",".",".","."]
,["3",".",".","5",".","6","2",".","."]
,[".",".","1","3",".","5",".","9","."]
,[".",".",".","8",".",".",".",".","."]
,[".","9",".",".",".",".","4",".","."]]



# Call the main function
if __name__ == '__main__':

    # Execution
    result = main(board_7)

    # Retrieve relevant information
    print('---------------------- Original Board -----------------------')
    print(result["original_board"])
    print('----------------------- Is it valid? ------------------------')
    print(result["valid_input"]["input_allowed"])
    if result["solved"] == True:
        print('----------------------- Board Solved ------------------------')
        print(result["solution"])
        print('------------------- Number of iterations --------------------')
        print(result["number_iterations"])
        print('------------------- History of the solution --------------------')
        for key, value in result["solution_history"].items():
            print("*********************")
            print(key)
            print("iteration_number", value["iteration_number"])
            print("step", value["step"])
            print(value["board_state"])

---------------------- Original Board -----------------------
[[ 1. nan nan nan  6. nan nan nan nan]
 [nan nan  3.  9. nan  1. nan  4. nan]
 [ 2. nan nan nan nan nan nan nan  7.]
 [nan nan nan nan  8. nan nan  5. nan]
 [nan nan  6. nan  4. nan nan nan nan]
 [ 3. nan nan  5. nan  6.  2. nan nan]
 [nan nan  1.  3. nan  5. nan  9. nan]
 [nan nan nan  8. nan nan nan nan nan]
 [nan  9. nan nan nan nan  4. nan nan]]
----------------------- Is it valid? ------------------------
True
----------------------- Board Solved ------------------------
[[1. 4. 8. 7. 6. 2. 9. 3. 5.]
 [6. 7. 3. 9. 5. 1. 8. 4. 2.]
 [2. 5. 9. 4. 3. 8. 1. 6. 7.]
 [9. 2. 7. 1. 8. 3. 6. 5. 4.]
 [5. 8. 6. 2. 4. 9. 3. 7. 1.]
 [3. 1. 4. 5. 7. 6. 2. 8. 9.]
 [4. 6. 1. 3. 2. 5. 7. 9. 8.]
 [7. 3. 2. 8. 9. 4. 5. 1. 6.]
 [8. 9. 5. 6. 1. 7. 4. 2. 3.]]
------------------- Number of iterations --------------------
4
------------------- History of the solution --------------------
*********************
move_1
iteration_number 1
step firs

### [External Solution](https://leetcode.com/problems/sudoku-solver/solutions/3869154/python-striver-s-solution-backtracking-easy/)

In [28]:
### Challenge 03
### Solve Sudoku
### External Solution



### Definition of the solution

# Class for the solution
class Solution:
    def solveSudoku(self, board: list[list[str]]) -> None:
        
        # Function to check whether it is safe to insert a number in board or not
        def safe(row,col,k,board):
            for i in range(9):
                # For checking element in same row
                if board[row][i]==k:
                    return False
                # For checking element in same column
                if board[i][col]==k:
                    return False
                # For checking element in same box
                if board[3*(row//3)+i//3][3*(col//3)+i%3]==k:
                    return False
            return True
        
        # Function to solve the board iteratively
        def solve(board):
            for i in range(9):
                for j in range(9):
                    if board[i][j]=='.':
                        for k in range(1,10):
                            k=str(k)
                            if safe(i,j,k,board):
                                board[i][j]=k
                                if solve(board):
                                    return True
                                board[i][j]='.'
                        # If it is not possible to insert any particular value at that cell
                        return False 
            # If all the values are filled then it will return True
            return True
        
        # Execution of the solution
        solve(board)
        return board

In [30]:
### Testing External Solution

# Initialize inputs
board_1 = [["5","3",".",".","7",".",".",".","."]
,["6",".",".","1","9","5",".",".","."]
,[".","9","8",".",".",".",".","6","."]
,["8",".",".",".","6",".",".",".","3"]
,["4",".",".","8",".","3",".",".","1"]
,["7",".",".",".","2",".",".",".","6"]
,[".","6",".",".",".",".","2","8","."]
,[".",".",".","4","1","9",".",".","5"]
,[".",".",".",".","8",".",".","7","9"]]
board_2 = [[".",".",".",".",".",".",".",".","."]
,["7",".",".",".","8",".",".",".","5"]
,["2",".",".","9",".",".","3",".","."]
,["9",".",".","7",".",".",".",".","8"]
,["4","7",".","3",".",".",".","5","."]
,[".",".",".","4",".",".",".",".","1"]
,["5",".",".",".","6",".",".",".","2"]
,[".","1","4",".",".",".",".",".","."]
,[".",".","2",".","9",".",".","1","."]]
board_3 = [["5","3",".",".","7",".",".",".","."]
,["6",".",".","1","9","5",".",".","."]
,[".","9","8",".",".",".",".","6","."]
,["8",".",".",".","6",".",".",".","3"]
,["4",".",".","8",".","3",".",".","1"]
,["7",".",".",".","2",".",".",".","6"]
,[".","6",".",".",".",".","2","8","."]
,[".",".",".","4","1","9",".",".","5"]
,[".",".",".",".","8",".",".","7","9"]]
board_4 = [["2","5",".",".",".",".",".",".","4"]
,[".",".",".",".","5",".",".",".","9"]
,[".","8",".","3",".",".","2","5","."]
,[".",".",".",".",".",".",".",".","2"]
,[".","3",".",".",".","7",".",".","."]
,["8",".",".",".","4",".","1","6","."]
,["1",".",".",".","6",".","5","8","."]
,[".",".",".",".",".",".",".","9","."]
,[".",".","6","4",".",".",".",".","."]]
board_5 = [[".","4",".",".","1","9",".","7","6"]
,["8",".",".",".",".",".",".",".","3"]
,[".",".",".","6",".",".",".",".","."]
,[".","9",".",".","2","7",".","1","."]
,[".",".","4",".",".",".","9",".","."]
,[".",".",".",".",".","5",".",".","."]
,[".",".","3",".","6","2",".",".","7"]
,[".","2",".","5",".",".",".",".","."]
,[".",".",".","4",".",".",".","6","."]]
board_6 = [[".",".","1",".",".","4","8",".","."]
,[".",".","3","2","8",".",".",".","5"]
,[".","2",".",".",".","6",".",".","."]
,[".",".","5",".",".",".",".","7","."]
,[".","3",".","9","1",".","6",".","."]
,[".",".",".",".",".","2",".",".","."]
,[".","9",".","8","3",".","1",".","."]
,["1",".",".",".",".",".",".",".","6"]
,[".",".",".",".","4",".",".",".","."]]
board_7 = [["1",".",".",".","6",".",".",".","."]
,[".",".","3","9",".","1",".","4","."]
,["2",".",".",".",".",".",".",".","7"]
,[".",".",".",".","8",".",".","5","."]
,[".",".","6",".","4",".",".",".","."]
,["3",".",".","5",".","6","2",".","."]
,[".",".","1","3",".","5",".","9","."]
,[".",".",".","8",".",".",".",".","."]
,[".","9",".",".",".",".","4",".","."]]

# Executions
# print('--------------------------------------------')
# print('Example 1')
# ext_result_1 = Solution.solveSudoku(Solution,board_1)
# print(ext_result_1)
# print('--------------------------------------------')
# print('Example 2')
# ext_result_2 = Solution.solveSudoku(Solution,board_2)
# print(ext_result_2)
# print('--------------------------------------------')
# print('Example 3')
# ext_result_3 = Solution.solveSudoku(Solution,board_3)
# print(ext_result_3)
# print('--------------------------------------------')
# print('Example 4')
# ext_result_4 = Solution.solveSudoku(Solution,board_4)
# print(ext_result_4)
# print('--------------------------------------------')
# print('Example 5')
# ext_result_5 = Solution.solveSudoku(Solution,board_5)
# print(ext_result_5)
# print('--------------------------------------------')
# print('Example 6')
# ext_result_6 = Solution.solveSudoku(Solution,board_6)
# print(ext_result_6)
print('--------------------------------------------')
print('Example 7')
ext_result_7 = Solution.solveSudoku(Solution,board_7)
print(ext_result_7)
print('--------------------------------------------')


--------------------------------------------
Example 7
[['1', '4', '8', '7', '6', '2', '9', '3', '5'], ['6', '7', '3', '9', '5', '1', '8', '4', '2'], ['2', '5', '9', '4', '3', '8', '1', '6', '7'], ['9', '2', '7', '1', '8', '3', '6', '5', '4'], ['5', '8', '6', '2', '4', '9', '3', '7', '1'], ['3', '1', '4', '5', '7', '6', '2', '8', '9'], ['4', '6', '1', '3', '2', '5', '7', '9', '8'], ['7', '3', '2', '8', '9', '4', '5', '1', '6'], ['8', '9', '5', '6', '1', '7', '4', '2', '3']]
--------------------------------------------


### Final Remarks

First of all, it's relevant to mention that results for the `board_7` are the same in the proposed and the external solution. This is verified below. A similar test can be followed using the other available inputs, i.e., `board_1`, `board_2`, `board_3`, `board_4`, `board_5`, and `board_6`, or any other Sudoku board you want to test.

In [57]:
# Convert the results from the proposed solution and the external solution to the same format
p_result = result["solution"]
e_result_aux = ext_result_7
e_result = np.zeros((9,9))
for ii in range(9):
    for jj in range(9):
        e_result[ii,jj] = e_result_aux[ii][jj]

# Compare results
compare_results = np.array_equal(p_result, e_result)
if compare_results:
    print('Results are the same')
else:
    print('Results are different')

Results are the same


The approaches for both solutions, proposed and external, is quite different. In the external solution, it is used a recursive algorithm to find the solution of the Sudoku board. The logic behind is purely computational. On the other hand, the proposed solution was built as a set of functions that mimics typical Sudoku movements that a person normally would do. Besides, together with the final result, the entire history of the solution is retrieved in the output. The entire explanation of proposed solution can be found [here](https://github.com/wgfajardom/portfolio/blob/develop/Challenges/03_solve_sudoku/explanation_solve_sudoku.md). 